## Hough Transform
* https://paper.dropbox.com/doc/Hough-Transform-KGkfnjuxH7hhHysOwuSFO

In [274]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys
p = os.path.join(os.path.dirname('__file__'), '..')
sys.path.append(p)
from common import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data

In [ ]:
DATA_DIR = '../data/'
json_fpath = os.path.join(DATA_DIR, 'volleyball_frame_00665.json')
img_fpath = os.path.join(DATA_DIR, 'volleyball_frame_00665.png')

## Helpers

In [ ]:
def plot_img(arr, fs=(10,10), cmap='gray', title=None):
    plt.figure(figsize=fs)
    plt.imshow(arr, cmap=cmap)
    plt.title(title)
    plt.show()
    
def load_img(fpath):
    return plt.imread(fpath)

def load_cv2_img(fpath, w=None, h=None, colorspace=None):
    img = cv2.imread(img_fpath)
    if colorspace is not None:
        img = cv2.cvtColor(img, colorspace)
    if None not in [w,h]:
        img = cv2.resize(img, (w, h), interpolation=cv2.INTER_CUBIC)
    return img

def make_boxes(meta):
    boxes = {}
    for idx,row in meta.iterrows():
        box = json.loads(row.to_json())
        fname = row['filename']
        if fname in boxes:
            boxes[fname].append(box)
        else:
            boxes[fname] = [box]
    return boxes

def threshold(img, color, thresh):
    """
    color = [b, g, r] or [r,b,g] or [h,s,v]
    thresh = margin allowed around color
    """
    min_color = np.array([color[0]-thresh, color[1]-thresh, color[2]-thresh])
    max_color = np.array([color[0]+thresh, color[1]+thresh, color[2]+thresh])
    min_color[min_color < 0] = 0
    max_color[max_color > 255] = 255
    print("Min", min_color)
    print("Max", max_color)
    
    mask = cv2.inRange(img, min_color, max_color)
    result = cv2.bitwise_and(img, img, mask=mask)
    return mask, result

## Erosion Dilation

In [ ]:
# RGB
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
plot_img(img, fs=(14,14), title="Original")

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
erosion = cv2.erode(img, kernel,iterations = 1)
plot_img(erosion, fs=(14,14), title="Erosion")

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4,4))
dilation = cv2.dilate(img, kernel,iterations = 2)
plot_img(dilation, fs=(14,14), title="Dilation")

img = cv2.dilate(erosion, kernel, iterations = 2)
plot_img(img, fs=(14,14), title="Erosion + Dilation")

## Edge Detection

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
gray = cv2.GaussianBlur(gray, ksize=(5,5), sigmaX=3)
plot_img(gray, fs=(14,14), title="Grayscale")

# use Canny edge detector to find edges in the image.  The thresholds determine how
# weak or strong an edge will be detected.  These can be tweaked.
lower_threshold = 25
upper_threshold = 25
edges = cv2.Canny(gray, lower_threshold, upper_threshold)
plot_img(edges, fs=(14,14))

# Mask top of image
edges[:425,:] = 0
edges[700:,:] = 0
edges[:,1150:] = 0
plot_img(edges, fs=(14,14))

## Hough Transform


* https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html
* https://docs.opencv.org/3.3.1/d9/db0/tutorial_hough_lines.html
* http://homepages.inf.ed.ac.uk/rbf/HIPR2/hough.htm
* Polar coords = (rho, theta)
* rho = distance/length of normal
* normal = perpendicular line (along target line) that intersects origin
* theta = angle created by intersection with normal
* Returns lines in parametric (polar coordinates) form (p, theta)
    * p = x cos(theta) + y sin(theta)
    * p = perpendicular distance from origin (0,0) - top left corner
    * theta = angle formed by p and horizontal axis (counter clockwise) (radians)
    * r is measured in pixels and 0 is measured in radians.
![Houghlines](https://docs.opencv.org/3.0-beta/_images/houghlines1.svg)
![hough](http://cdncontribute.geeksforgeeks.org/wp-content/uploads/Hough_transform_diagram.png)

* https://www.geeksforgeeks.org/line-detection-python-opencv-houghline-method/
* Applications of Hough transform:

    * It is used to isolate features of a particular shape within an image.
    * Tolerant of gaps in feature boundary descriptions and is relatively unaffected by noise.
    * Used extensively in barcode scanning, verification and recognition
* Steps
    * First parameter, Input image should be a binary image, so apply threshold edge detection before finding applying hough transform.
    * Second and third parameters are r and θ(theta) accuracies respectively.
    * Fourth argument is the threshold, which means minimum vote it should get for it to be considered as a line.
    * Remember, number of votes depend upon number of points on the line. So it represents the minimum length of line that should be detected.
   
* ρ = x cos θ + y sin θ

* where:
    * ρ (rho) = distance from origin to the line. [-max_dist to max_dist].
          max_dist is the diagonal length of the image.  
    * θ = angle from origin to the line. [-90° to 90°]

In [ ]:
# Higher threshold means a line needs to be stronger to be detected
thresh = 150
lines = cv2.HoughLines(edges, 1, np.pi / 180, thresh)
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)

# convert polor coordinates to x,y pixel coordinates
# origin (0,0) is top left in numpy/cv2
# p = distance from origin (perpendicular to hough line)
# theta = angle in radians formed by perpendicular line
coords = []
for line in lines:
    for rho, theta in line:
        degrees = theta * (180/math.pi)
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 1000 * -b)
        y1 = int(y0 + 1000 * a)
        x2 = int(x0 - 1000 * -b)
        y2 = int(y0 - 1000 * a)

        if degrees > 145:
            orient = 'right_vertical'
        elif degrees < 45:
            orient = 'left_vertical'
        elif degrees > 85 and degrees < 95:
            orient = 'horizontal'
        else:
            orient = 'other'
            
        coords.append([x1,y1,x2,y2,rho,theta,degrees,orient])

        img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
        cv2.line(img, pt1=(x1, y1), pt2=(x2, y2), color=(0, 0, 255), thickness=3)
        plot_img(img, fs=(18,18), title=coords[-1])

# write the image to disk
plot_img(img, fs=(18,18))

### Hough Lines Probabalistic

In [ ]:
# Hough Transform - Lines

x_delta = 100
y_delta = 100
min_line_length = 400
max_line_gap = 1500
thresh = 100
#minLineLength - Minimum length of line
#maxLineGap - Maximum allowed gap between line segments to treat them as single line.
lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180,
                        threshold=thresh, maxLineGap=max_line_gap, 
                        minLineLength=min_line_length)
line_coords = []

for line in lines:
    x1,y1,x2,y2 = line[0]
    slope = (y2-y1) / (x2-x1)
    print(x1,y1,x2,y2,slope)
    if abs(x2 - x1) < x_delta:
        orient = 'vertical'
    elif abs(y2 - y1) < y_delta:
        orient = 'horizontal'
    else:
        orient = 'other' #None
    coords = [x1,y1,x2,y2,slope,orient]
    line_coords.append(coords)
    img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
    cv2.line(img, pt1=(x1, y1), pt2=(x2, y2), color=(0, 0, 255), thickness=3)
    plot_img(img, fs=(18,18), title=coords)

## Cleaning up Hough Lines

* https://campushippo.com/lessons/detect-highway-lane-lines-with-opencv-and-python-21438a3e2
* How to clean up the lines, extend them, and average nearby lines?
* https://stackoverflow.com/questions/44449871/fine-tuning-hough-line-function-parameters-opencv
* Since the lines are mostly vertical and horizontal, you can easily split the lines based on their position. If the two y-coordinates of one line are near each other, then the line is mostly horizontal. If the two x-coordinates are near each other, then the line is mostly vertical. So you can segment your lines into vertical lines and horizontal lines that way.
* https://alyssaq.github.io/2014/understanding-hough-transform/

### Detecting Horizontal / Vertical Lines

In [ ]:
"""
r = negative when base of normal is above the top of image (negative y value)
y1,y2 = negative when base of normal is above top of image
theta = between 0 and 90 when the base of the normal is inside the image
theta > 90 when base of normal is outside the imag

Horizontal lines
   * Angles close to 90
   
Vertical lines
    * Angles close to 180/0
    
A volleyball court viewed straight on should have
    * 2 horizontal lines
    * 1 vertical line with angle close to 0
    * 1 vertical line with angle close to 180
    
Taking a look at the image, we can set some sensible defaults

vertical_left = angle < 45 degres
vertical_right = angle > 145 degrees
horizonal lines = angle between 85 and 95
"""

for coord in coords:
    if coord[-2] > 145:
        print('right vertical')
    elif coord[-2] < 45:
        print('left vertical')
    elif coord[-2] > 85 and coord[-2] < 95:
        print('horizontal')
    else:
        print('other')

### Extending Hough Lines

In [ ]:
height,width,c = img.shape
new_coords = []
for coord in coords:
    x1,y1,x2,y2 = coord[:4]
    slope = (y2-y1) / (x2-x1)
    #y = mx + b
    b = y1 - slope * x1
    # x intercepts
    x1_int = int(-b / slope) # -b/m = 0 = mx + b), x when y == 0
    x2_int = int((height - b) / slope) #x when y == height
    # y intercepts
    y1_int = int(b) #y when x == 0
    y2_int = int(slope * width + b) #y when x == width
    details = {
        'x1':x1,
        'y1':y1,
        'x2':x2,
        'y2':y2,
        'x1_intercept': x1_int,
        'y1_intercept': y1_int,
        'x2_intercept': x2_int,
        'y2_intercept': y2_int,
        'slope': slope,
        'b': b,
        'orient': coord[-1]
    }
    new_coords.append(details)
    img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
    if details['orient'] == 'horizontal':
        cv2.line(img, pt1=(0, y1_int), pt2=(width, y2_int), 
                 color=(0, 0, 255), thickness=3)
    else:
        cv2.line(img, pt1=(x1_int, 0), pt2=(x2_int, height), 
             color=(0, 0, 255), thickness=3)
    plot_img(img, fs=(18,18), title=details)

## Finding the Intersection of Lines

* Cross Product of two vectors gives the intersection
* Steps:
    * Given two points, find the line
    * Find the slope
    * Find the b intercept
    * Convert to standard form (Ax + By + C = 0)
    * Convert to vector form ([A B C])
    * Compute the cross product
* Cross product
    * https://paper.dropbox.com/doc/Vectors-ehJskFyt7xGIdePpamfhu 
    * https://www.symbolab.com/solver/vector-cross-product-calculator
* https://stackoverflow.com/questions/44449871/fine-tuning-hough-line-function-parameters-opencv

### Intersection w Cross Product

In [ ]:
def vector_from_points(x1,y1,x2,y2):
    m = (y2-y1) / (x2-x1)
    b = y1 - (m * x1)
    # Ax + By + C
    #1y - mx - b = 0
    v = np.array([-m, 1, -b])
    return v

def intersection_two_points(A, B):
    Av = vector_from_points(A['x1'], A['y1'], A['x2'], A['y2'])
    Bv = vector_from_points(B['x1'], B['y1'], B['x2'], B['y2'])
    cross = np.cross(Av, Bv)
    cross /= cross[2]
    return int(cross[0]), int(cross[1])

A = {'x1':0, 'y1':4, 'x2':4, 'y2':0}
B = {'x1':0, 'y1':0, 'x2':4, 'y2':4}
intersection_two_points(A, B)

### Include horizontal and vertical line intersections only

In [ ]:
def find_intersecting_points(lines):
    horizontal_lines = [l for l in lines if 'horizontal' in l['orient']]
    vertical_lines = [l for l in lines if 'vertical' in l['orient']]
    intersecting_points = []
    for hl in horizontal_lines:
        hp = {'x1':hl['x1'], 'y1':hl['y1'], 'x2':hl['x2'], 'y2':hl['y2']}
        for vl in vertical_lines:
            vp = {'x1':vl['x1'], 'y1':vl['y1'], 'x2':vl['x2'], 'y2':vl['y2']}
            intersection = intersection_two_points(hp, vp)
            intersecting_points.append(intersection)
    return intersecting_points

In [ ]:
intersecting_points = find_intersecting_points(new_coords)
intersecting_points

In [ ]:
# PLot
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
for point in intersecting_points:
    cv2.circle(img, point, 1, color=(255,0,0), thickness=5)
    
plot_img(img, fs=(18,18), title=None)

### Cluster points with K-Means

In [ ]:
points = np.array(intersecting_points)
print(points.shape)
def cluster_points(points, n_clusters):
    points = np.float32(points)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, _, centers = cv2.kmeans(points, n_clusters, None, criteria, 10, cv2.KMEANS_PP_CENTERS)
    return centers.astype(int)
centers = cluster_points(points, 4)

In [ ]:
centers

In [ ]:
# PLot
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
for c in centers:
    cv2.circle(img, (c[0],c[1]), 1, color=(255,0,0), thickness=5)
    
plot_img(img, fs=(18,18), title=None)

## Segment court

* https://stackoverflow.com/questions/17241830/opencv-polylines-function-in-python-throws-exception/18817152#18817152

In [ ]:
y_sorted = centers[centers[:,1].argsort()]

# Top left and right
top = y_sorted[:2]
top = top[top[:,0].argsort()]
top_left = top[0]
top_right = top[1]

# Bottom left and right
bottom = y_sorted[2:]
bottom = bottom[bottom[:,0].argsort()]
bottom_left = bottom[0]
bottom_right = bottom[1]

ordered_points = [top_left,top_right,bottom_left,bottom_right]

In [ ]:
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
masked_img = cv2.fillPoly(img, np.array([centers.tolist()]), color=(255,255,255))
plot_img(masked_img)

In [ ]:
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
img = cv2.polylines(img, np.array([centers.tolist()]), isClosed=1, color=(255,255,255), thickness=3)

In [ ]:
plot_img(img)